# 適切な軸設定でクロス集計表を作成し，属性間のデータの偏りを把握する(2022/02/23)
---

## 概要
---
本notebookではピボットテーブルと，その特殊な形式であるクロス集計表について知り，実際にPython(主にpandas)を用いたシミュレーションを行う．

## ピボットテーブル(Pivot table)
---

ピボットテーブルは，縦軸と横軸にそれぞれ別の要素を配置して集計した表形式のデータである．例えば，縦軸に性別，横軸に学校名を割り当てて，それぞれの高校の平均身長を要素とすれば，ピボットテーブルの一種と言える．

また，ピボットテーブルの集計する値が頻度・度数である場合，それはクロス集計表と呼ばれる．

適切な軸設定についてであるが，これは調査目的によるだろう．先述の平均身長の例で言えば，性別の平均身長を把握したいのに，縦軸に学年，横軸に学校名を割り当てるのは適切でない．

言葉で説明してもイメージがわかないので，ここからは実際にpandasを用いてピボットテーブルを作成していく．便利なことに，pandasライブラリではピボットテーブルを作成する関数(pivot_table)と，クロス集計を行う関数(crosstab)が実装されている．

## Pythonを用いたシミュレーション(ピボットテーブルの作成)
---

今回はtips datasetを用いてピボットテーブルを作成する．

In [1]:
# tips.csvの読み込み
import pandas as pd

tips_df = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv',   
)
tips_df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


このようにtips datasetでは，食事に来た客がどれだけのチップを支払ったかが，客の属性と共に記されている．

ここで階層型インデックスとして「day(曜日)」と「smoker(喫煙の有無)」を指定して，ピボットテーブルを作成してみる．

In [2]:
# ピボットテーブルを作成
tips_df.pivot_table(index=['day', 'smoker'])

size       tip  total_bill
day  smoker                                
Fri  No      2.250000  2.812500   18.420000
     Yes     2.066667  2.714000   16.813333
Sat  No      2.555556  3.102889   19.661778
     Yes     2.476190  2.875476   21.276667
Sun  No      2.929825  3.167895   20.506667
     Yes     2.578947  3.516842   24.120000
Thur No      2.488889  2.673778   17.113111
     Yes     2.352941  3.030000   19.190588

pivot_table関数はデフォルトで平均(mean)を計算する．

このピボットテーブルから，例えば金曜日(Fri)に来た非喫煙者(No)の客は，

- 平均2人強のグループ
- チップは2.8USドルくらい
- 食費は18.4USドルくらい

といったことが分かる．

もうひとつ，軸を変えてピボットテーブルを作ってみよう．

In [3]:
# 軸として曜日と性別を設定
tips_df.pivot_table(index=['day', 'sex'])

size       tip  total_bill
day  sex                                   
Fri  Female  2.111111  2.781111   14.145556
     Male    2.100000  2.693000   19.857000
Sat  Female  2.250000  2.801786   19.680357
     Male    2.644068  3.083898   20.802542
Sun  Female  2.944444  3.367222   19.872222
     Male    2.810345  3.220345   21.887241
Thur Female  2.468750  2.575625   16.715312
     Male    2.433333  2.980333   18.714667

食費の総額について，女性より男性の方が多い傾向にあり，休日の方が多い傾向にもある，といったことが見て取れる．

また，パラメータmarginsにTrueを設定すると，小計も計算してくれる．

In [4]:
# 先ほどと同じ軸で小計も表示する
tips_df.pivot_table(index=['day', 'sex'], margins=True)

size       tip  total_bill
day  sex                                   
Fri  Female  2.111111  2.781111   14.145556
     Male    2.100000  2.693000   19.857000
Sat  Female  2.250000  2.801786   19.680357
     Male    2.644068  3.083898   20.802542
Sun  Female  2.944444  3.367222   19.872222
     Male    2.810345  3.220345   21.887241
Thur Female  2.468750  2.575625   16.715312
     Male    2.433333  2.980333   18.714667
All          2.569672  2.998279   19.785943

### クロス集計表の作成

縦軸と横軸に頻度データの要素を指定すれば，そのピボットテーブルはクロス集計表になる．

また，関数pivot_tableは用いる関数を指定できるので，lenあるいはcountを指定すればよい．

In [5]:
# クロス集計表の作成
tips_df[['time', 'smoker', 'day', 'size']].pivot_table(index=['time', 'smoker'], columns=['day'], aggfunc=len, margins=True, fill_value=0)

size                  
day            Fri Sat Sun Thur  All
time   smoker                       
Dinner No        3  45  57    1  106
       Yes       9  42  19    0   70
Lunch  No        1   0   0   44   45
       Yes       6   0   0   17   23
All             19  87  76   62  244

なお，クロス集計表の作成にはcrosstabメソッドが便利

In [6]:
# クロス集計表の作成
pd.crosstab(index=[tips_df.time, tips_df.smoker], columns=[tips_df.day], margins=True)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

## 参考文献
---

[seaborn_tips_dataset，Kaggle](https://www.kaggle.com/ranjeetjain3/seaborn-tips-dataset)

[seaborn-data/tips.csv，github](https://github.com/mwaskom/seaborn-data/blob/master/tips.csv)

[pandas.read_csv，pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

[pandas.pivot_table，pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html)

[pandas.crosstab，pandas 1.4.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html)